In [1]:
import numpy as np
import time
import qcodes
import pyvisa as visa
# enable to get a more debuggable output
#visa.log_to_screen()

from qcodes.instrument_drivers.yokogawa.GS200 import GS200
#visapy-py resource manager
py_rm = visa.ResourceManager('@py')
#national instruments resource manager
ni_rm = visa.ResourceManager()

print(py_rm)
print(ni_rm)

/home/nikolai/miniconda3/envs/qcodes/lib/python3.9/site-packages/gpib_ctypes/gpib/gpib.py:54: UserWarning: GPIB library not found. Please manually load it using _load_lib(filename). All GPIB functions will raise OSError until the library is manually loaded.
  warnings.warn(message)


Resource Manager of Visa Library at py
Resource Manager of Visa Library at /usr/lib/x86_64-linux-gnu/libvisa.so.21.5.0


In [2]:
# for some reason, the USB access has to be initialized with the python resource manager
# the resource is opened to be recognizable by the ni drivers
py_rm.open_resource(py_rm.list_resources()[1])

print("visa-py: " + str(py_rm.list_resources()))
print("visa-ni: " + str(ni_rm.list_resources()))

visa-py: ('ASRL/dev/ttyS4::INSTR', 'USB0::2849::57::91LA25023::0::INSTR')
visa-ni: ('USB0::0x0B21::0x0039::91LA25023::INSTR', 'ASRL5::INSTR')


In [3]:
# intialize the instrument with the following parameters
gs_addr = ni_rm.list_resources()[0]
gs = GS200("gs200", address=gs_addr, terminator="\n")

Connected to: YOKOGAWA GS210 (serial:91LA25023, firmware:1.05) in 0.23s


In [4]:
# can store a QCoDeS snapshot of the instrument with the following command
#gs.snapshot()

In [4]:
# resetting the instrument
gs.reset()
gs.auto_range(True)

In [6]:
# the output mode of the instrument
gs.source_mode()

'VOLT'

In [9]:
gs.voltage(0.1)
gs.output('on')

In [21]:
gs.output('off')

In [7]:
# automatically adjust the range of the outputs
gs.auto_range(True)

In [8]:
gs.source_mode('CURR')
gs.source_mode()

'CURR'

In [9]:
gs.current(0)
gs.output('off')
gs.output()

'off'

In [41]:
import ipywidgets as widgets
from IPython.display import display
mode_toggle = widgets.ToggleButtons(
    options=['Voltage', 'Current'],
    description='Mode:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    #tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

mode_toggle

ToggleButtons(description='Mode:', options=('Voltage', 'Current'), value='Voltage')

In [22]:
mode_toggle.options

('Voltage', 'Current')

In [25]:
mode_toggle.value[:4].upper()

'VOLT'

In [42]:
def changed(change):
    print(mode_toggle.value)
    gs.source_mode(mode_toggle.value[:4].upper())
    
def on_reset_clicked(b):
    with output:
        gs.reset()
    
mode_toggle.observe(changed, 'value')

Button(description='Reset!', style=ButtonStyle())

Output()

In [44]:
# ramping up current from 0 to 100mA in intervals of 0.1mA each 0.01 seconds
gs.output('on')
gs.current(0)
t1 = time.time()
gs.ramp_current(0.1, 0.0001, 0.01)
t2 = time.time()
print("Ramping took {:.4} seconds".format(t2-t1))

Ramping took 11.7 seconds


In [16]:
# print a readable, more easy to understand snapshot of the data
gs.print_readable_snapshot()

gs200:
	parameter    value
--------------------------------------------------------------------------------
BNC_in        :	None 
BNC_out       :	None 
IDN           :	{'vendor': 'YOKOGAWA', 'model': 'GS210', 'serial': '91LA25023...
auto_range    :	True 
current       :	0.1 (I)
current_limit :	None (I)
current_range :	0.1 (I)
four_wire     :	None 
guard         :	None 
line_freq     :	None (Hz)
output        :	on 
output_level  :	0.1 (I)
range         :	0.1 (I)
source_mode   :	CURR 
system_errors :	None 
timeout       :	5 (s)
voltage_limit :	None (V)


ValueError: max() arg is an empty sequence

In [ ]:
# disconnect from the instrument, it will have to reinitialized again
gs.close()